# Imports

In [3]:
%run Imports.ipynb
name = 'Kred'

# specific imports
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV


key_featsubgroups = pd.read_pickle('../pickle/2_FS/' + name + '/key_featsubgroups.pkl')
df = pd.read_pickle('../pickle/2_FS/' + name + '/2_df_new_.pkl')

/home/RDC/fliegebe/.conda/envs/Work/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(df[target].value_counts()/df.shape[0])
print('df_shape: ', df.shape)

arrears
1   0.646
0   0.354
Name: count, dtype: float64
df_shape:  (129457, 418)


In [5]:
# param grid scikit
param_grid = {
    'hidden_layer_sizes': [(h,) for h in [5, 10, 20, 50, 100, 200, 300, 406]],
    'activation': ['relu', 'tanh', 'logistic'],                   # activation functions
    'solver': ['adam', 'sgd'],                                    # optimization algorithms
    'alpha': [0.0001, 0.001, 0.01, 0.1],                                # L2 regularization
    'learning_rate': ['constant', 'adaptive'],                    # learning rate strategy
    'learning_rate_init': [0.001, 0.005, 0.01, 0.05],                           # initial learning rate
    'max_iter': [100, 200, 500, 750, 1000]                                        # training epochs
}

In [6]:
# testing grid
pg = {
    'hidden_layer_sizes': [(h,) for h in [5,10]],
    'activation': ['relu'],                   # activation functions
    'solver': ['adam'],                                    # optimization algorithms
    'alpha': [0.1],                                # L2 regularization
    'learning_rate': ['constant'],                    # learning rate strategy
    'learning_rate_init': [0.05],                           # initial learning rate
    'max_iter': [10, 15]                                        # training epochs
}

In [7]:
with open('../pickle/4_Model_Optuna/results_dict_updated.pkl', 'rb') as f:
    results_dict = pickle.load(f)

In [8]:
with open('../pickle/3_Model/xgb_default_params.pkl', 'rb') as f:
    default_params = pickle.load(f)


# 2) Create Model prediction functions

## 2.1) Split dataset into train/testing while excluding demographic features

In [9]:
def split_data_4(df, key_featsubgroups=key_featsubgroups, target=target, test_size=0.2, random_state=42):
    """
    Splits the dataset into training and testing sets while excluding demographic features.

    Parameters:
    df (DataFrame): The dataset containing features and target variable.
    key_featsubgroups (DataFrame): A mapping of feature subgroups.
    target (str): The name of the target variable.
    test_size (float, optional): The proportion of the dataset to allocate for testing. Default is 0.2.
    random_state (int, optional): Random seed for reproducibility. Default is 42.

    Returns:
    tuple: X_train, X_test, y_train, y_test (training and testing datasets)
    """

    # Extract demographic features
    demo_feat = key_featsubgroups.loc[key_featsubgroups['subgroup'] == 'demo', 'list_features'].values[0]
    print("Demographic Features:", demo_feat)

    # Separate features (X) and target variable (y), excluding demographic features
    X = df.drop(columns=[target] + demo_feat)
    y = df[target]

    # Split the dataset into training (80%) and testing (20%) sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Print dataset shapes
    print(f"Training Features Shape: {X_train.shape}")
    print(f"Training Labels Shape: {y_train.shape}")
    print(f"Testing Features Shape: {X_test.shape}")
    print(f"Testing Labels Shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

## 2.2) Model training and prediction

In [10]:
x_train, x_test, y_train, y_test = split_data_4(df)

Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']
Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)


## 7) Neural Network Sklearn

In [11]:
def sklearn_nn(param_grid, df, results_dict):
    """
    Tunes and trains an MLPClassifier (one hidden layer) via Optuna,
    evaluates performance, and stores results in the given dictionary.

    Parameters:
    -----------
    default_params : dict
        Default hyperparameters (each as a list) for reference.
    param_grid : dict
        Search space for Optuna (keys are param names, values are lists).
    df : DataFrame
        Dataset containing features and the target.
    results_dict : dict
        Dictionary to store model performance metrics.

    Returns:
    --------
    best_params : dict
        Best parameters found by Optuna.
    results_dict : dict
        Updated results dictionary containing model evaluation metrics.
    """
    start_time = time.time()

    # split_data_4 must return X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = split_data_4(df)

    # total number of trials = sum of lengths of all search lists
    n_trials = int(np.sum([len(v) for v in param_grid.values()]))
    print(f"No. of trials scheduled: {n_trials}")

    def objective(trial):
        # sample each hyperparameter categorically
        params = {
            key: trial.suggest_categorical(key, values)
            for key, values in param_grid.items()
        }
        clf = MLPClassifier(**params, random_state=0)
        # use 5-fold CV accuracy as objective
        return cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy').mean()

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params
    print("Best hyperparameters found:", best_params)

    # Train final model with best params
    clf_best = MLPClassifier(**best_params, random_state=0)
    clf_best.fit(X_train, y_train)

    # Cross-validated predictions on train set
    y_train_pred = cross_val_predict(clf_best, X_train, y_train, cv=5)
    # Predictions on test set
    y_test_pred = clf_best.predict(X_test)

    # Compute confusion matrices
    cfm_train = confusion_matrix(y_train, y_train_pred)
    cfm_test  = confusion_matrix(y_test,  y_test_pred)

    # Compute accuracy
    acc_train = accuracy_score(y_train, y_train_pred)
    acc_test  = accuracy_score(y_test,  y_test_pred)

    # Compute F1-scores for both classes
    f1_train_1 = f1_score(y_train, y_train_pred, pos_label=1)
    f1_train_0 = f1_score(y_train, y_train_pred, pos_label=0)
    f1_test_1  = f1_score(y_test,  y_test_pred,  pos_label=1)
    f1_test_0  = f1_score(y_test,  y_test_pred,  pos_label=0)

    # Compute ROC-AUC on test set
    y_test_proba = clf_best.predict_proba(X_test)[:, 1]
    roc_auc_test = roc_auc_score(y_test, y_test_proba)

    total_time = (time.time() - start_time) / 60
    print(f"MLP optimization & training completed in {total_time:.2f} minutes")

    # Store everything in results_dict
    results_dict["sklearn_nn"] = {
        "classifier": deepcopy(clf_best),
        "cfm_train": cfm_train,
        "cfm_test":  cfm_test,
        "train_accuracy": acc_train,
        "test_accuracy":  acc_test,
        "train F1-score label 1": f1_train_1,
        "train F1-score label 0": f1_train_0,
        "test F1-score label 1":  f1_test_1,
        "test F1-score label 0":  f1_test_0,
        "test roc auc score":     roc_auc_test,
        "default_params": default_params,
        "param_grid":    param_grid,
        "best_params":   best_params,
        "time_m":        total_time
    }

    return best_params, results_dict

In [ ]:
best_params, results_dict = sklearn_nn(pg, df, results_dict)

Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']


[I 2025-06-10 15:48:54,529] A new study created in memory with name: no-name-6399edb1-329f-4893-86b8-bf3fab3c6d91


Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)
No. of trials scheduled: 9


[I 2025-06-10 15:49:12,181] Trial 0 finished with value: 0.6394921064066046 and parameters: {'hidden_layer_sizes': (10,), 'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.05, 'max_iter': 10}. Best is trial 0 with value: 0.6394921064066046.
[I 2025-06-10 15:49:37,808] Trial 1 finished with value: 0.6443103365036451 and parameters: {'hidden_layer_sizes': (10,), 'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.05, 'max_iter': 15}. Best is trial 1 with value: 0.6443103365036451.
[I 2025-06-10 15:49:56,106] Trial 2 finished with value: 0.6394921064066046 and parameters: {'hidden_layer_sizes': (10,), 'activation': 'relu', 'solver': 'adam', 'alpha': 0.1, 'learning_rate': 'constant', 'learning_rate_init': 0.05, 'max_iter': 10}. Best is trial 1 with value: 0.6443103365036451.


In [ ]:
with open('../pickle/7_Model/sklearn_nn_results.pkl', 'wb') as f:
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('../pickle/7_Model/sklearn_nn_best_params.pkl', 'wb') as f:
    pickle.dump(best_params, f, pickle.HIGHEST_PROTOCOL)